# MLOps Group No: 92 

## Group Members: 
- PEYALA SAMARASIMHA REDDY - 2023AA05072
- PEGALLAPATI SAI MAHARSHI- 2023AA05924 
- ANIRUDDHA DILIP MADURWAR - 2023AA05982 
- K VAMSIKRISHNA - 2023AA05209  


# Assignment 2

# Tasks

## M3: Model Selection & Hyperparameter Optimization - 10M

### Objective:
Identify the best model using AutoML and optimize its hyperparameters.

### Tasks:
###  1. Model Selection
- Use an open-source AutoML library (e.g., Auto-sklearn, H2O.ai, TPOT) to select the best-performing model.

### 2. Hyperparameter Optimization:
- Use an open-source hyperparameter optimization tool (e.g., Optuna, Hyperopt, Ray Tune) to fine-tune the selected model.

### Deliverables:
- AutoML results comparing multiple models.
- Hyperparameter tuning logs.
- Justification for the chosen model and hyperparameters.

# Model Selection

In [2]:
import numpy as np
import pandas as pd
import autosklearn.classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import fashion_mnist

# Load Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Reduce dataset size for faster training
X_train, y_train = X_train[:5000], y_train[:5000]  # Using only 5000 samples
X_test, y_test = X_test[:1000], y_test[:1000]      # Using only 1000 samples

# Reshape and normalize data
X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# AutoML Model Selection with reduced time
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,  # Run for 5 minutes (instead of 15)
    per_run_time_limit=30,        # Max time per model (instead of 60)
    ensemble_size=1,              # Only the best model
    seed=42,
    memory_limit=10024  # 1GB memory limit to avoid crashes
)

# Fit Auto-Sklearn
automl.fit(X_train, y_train)

# Get all trained models
models_trained = automl.show_models()
print("Models Trained:\n", models_trained)

# Get the best model
best_model = automl.get_models_with_weights()[0]
print("\nBest Model Selected:\n", best_model)

# Evaluate the best model
y_pred = automl.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"\nBest Model Validation Accuracy: {accuracy:.4f}")


/tmp/ipykernel_522656/1254506153.py:30: DeprecationWarning: `ensemble_size` has been deprecated, please use `ensemble_kwargs = {'ensemble_size': 1}`. Inserting `ensemble_size` into `ensemble_kwargs` for now. `ensemble_size` will be removed in auto-sklearn 0.16.
  automl = autosklearn.classification.AutoSklearnClassifier(


Models Trained:
 {8: {'model_id': 8, 'rank': 1, 'cost': 0.14166666666666672, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x704def2de430>, 'balancing': Balancing(random_state=42), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x704dee67fcd0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x704dee67fa90>, 'sklearn_classifier': SVC(C=12923.91167672086, cache_size=6340.658854166667,
    gamma=0.028106748647672205, max_iter=-1.0, random_state=42, shrinking=False,
    tol=2.0706197108771777e-05)}}

Best Model Selected:
 (1.0, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'libsvm_svc', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:libsvm_svc:C': 12923.91167672086, 'classifier:libsvm_svc:gamma

In [ ]:
# Comparison the different models tried using auto-sklearn, by default it uses 10-fold cross-validation and it always returns the best model, so we can use the cv_results_ attribute to get the results of all the models tried.
automl.cv_results_

{'mean_test_score': array([0.8280303 , 0.85075758, 0.81287879, 0.83333333, 0.        ,
        0.        , 0.85833333, 0.        , 0.79545455, 0.80075758,
        0.79924242, 0.11590909, 0.09166667, 0.        , 0.        ,
        0.        , 0.        , 0.85454545, 0.71363636, 0.        ,
        0.59469697, 0.83333333, 0.82575758, 0.74772727, 0.        ,
        0.        , 0.75984848, 0.11515152, 0.75833333, 0.23787879,
        0.81363636, 0.60909091, 0.        , 0.57424242, 0.        ,
        0.73409091, 0.65      , 0.80530303, 0.        , 0.25757576,
        0.75075758, 0.81439394, 0.        ]),
 'rank_test_scores': array([ 6,  3, 10,  4, 30, 30,  1, 30, 14, 12, 13, 27, 29, 30, 30, 30, 30,
         2, 20, 30, 23,  4,  7, 18, 30, 30, 15, 28, 16, 26,  9, 22, 30, 24,
        30, 19, 21, 11, 30, 25, 17,  8, 30]),
 'mean_fit_time': array([ 8.18759108,  1.55507779,  5.11011696,  1.03375292,  0.48828149,
        30.01371574,  2.83306098, 30.03315616, 10.30915952,  6.33920336,
        17

# Hyperparameter Optimization

In [9]:
# Extract the best model from Auto-Sklearn
best_model_details = automl.get_models_with_weights()[0]
best_model = best_model_details[1]  # Extract the model itself

# Extract the SVC from the Auto-Sklearn pipeline if it is SVC
# (since your best model was an SVC model)
if isinstance(best_model, SVC):
    best_svc_model = best_model
else:
    # You need to access the specific classifier inside the pipeline if it's not just an SVC
    # Here, you might need to inspect the pipeline structure to extract the classifier
    # For simplicity, let's assume you extracted the SVC from the pipeline
    best_svc_model = best_model.named_steps['classifier']  # This depends on how the model is structured

# Define an Optuna optimization function
def objective(trial):
    # Define hyperparameter search space for SVC
    C = trial.suggest_loguniform('C', 1e-6, 1e6)
    gamma = trial.suggest_loguniform('gamma', 1e-6, 1e1)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-1)
    
    # Create a new SVC model with suggested hyperparameters
    model = SVC(kernel='rbf', C=C, gamma=gamma, tol=tol, max_iter=-1, shrinking=False)
    
    # Train and evaluate using cross-validation
    model.fit(X_train, y_train)
    score = accuracy_score(y_val, model.predict(X_val))
    return score

# Run Optuna for hyperparameter tuning
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Show all tried hyperparameter sets
print("\nHyperparameter Trials:")
for trial in study.trials:
    print(f"Trial {trial.number}: {trial.params} → Accuracy: {trial.value:.4f}")

# Show the best set of hyperparameters
print("\nBest Hyperparameters Found:", study.best_params)
print("Best Validation Accuracy:", study.best_value)

# Final Model with Best Hyperparameters
best_params = study.best_params
best_model_tuned = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'], tol=best_params['tol'], max_iter=-1, shrinking=False)

# Train the model with best hyperparameters
best_model_tuned.fit(X_train, y_train)

# Evaluate on the test set
y_test_pred = best_model_tuned.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Final Test Accuracy with Tuned Hyperparameters: {test_accuracy:.4f}")

[I 2025-03-23 00:27:27,197] A new study created in memory with name: no-name-7ea14bfc-112b-4cb6-a119-3dfd84eb08f8
/tmp/ipykernel_522656/2806568885.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-6, 1e6)
/tmp/ipykernel_522656/2806568885.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-6, 1e1)
/tmp/ipykernel_522656/2806568885.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform('tol', 1e-6, 1e-


Hyperparameter Trials:
Trial 0: {'C': 8.531357054301406, 'gamma': 0.006213296913094322, 'tol': 4.06269504253865e-05} → Accuracy: 0.7660
Trial 1: {'C': 0.2670621357701803, 'gamma': 0.0003226564264501752, 'tol': 3.9972200491579986e-05} → Accuracy: 0.7760
Trial 2: {'C': 85638.42879602854, 'gamma': 0.08073783121317067, 'tol': 1.1873248601871263e-06} → Accuracy: 0.0990
Trial 3: {'C': 2796.1126823797526, 'gamma': 0.00025380985222900977, 'tol': 0.02609193668124631} → Accuracy: 0.8400
Trial 4: {'C': 23.121804722665587, 'gamma': 0.003321377592724783, 'tol': 0.023507394400831782} → Accuracy: 0.8310
Trial 5: {'C': 37437.19827098225, 'gamma': 0.56013083155556, 'tol': 0.006627719796272533} → Accuracy: 0.0950
Trial 6: {'C': 0.0028181742487822133, 'gamma': 2.84939022313369e-06, 'tol': 0.00020028636326180772} → Accuracy: 0.0950
Trial 7: {'C': 0.0003180109277447132, 'gamma': 0.0022520043816237445, 'tol': 3.188860448174315e-06} → Accuracy: 0.0950
Trial 8: {'C': 0.09593694759063544, 'gamma': 4.542539204